In [2]:
from nltk.translate.bleu_score import sentence_bleu
from nltk import word_tokenize
import json
from utils import write_jsonl


In [16]:
def calculate_bleu(candidate, reference):
    '''
    candidate, reference: generated and ground-truth sentences
    '''
    weights = [
         (1./2., 1./2.),
         (1./3., 1./3., 1./3.),
         (1./4., 1./4., 1./4., 1./4.)
    ]
    reference = reference.strip().split()
    candidate = candidate.strip().split()
    score = sentence_bleu([reference], candidate, weights=weights)
    return score

In [22]:
llama_output_path = '/Users/raunaksood/Desktop/RAG/deepseek7B_results_bleu.jsonl'
save_path = '/Users/raunaksood/Desktop/RAG/deepseek7B_results_bleu_NEW.jsonl'
with open(llama_output_path) as f:
    llama_outputs = [json.loads(line) for line in f]

responses = [llama_outputs[i]['response'] for i in range(len(llama_outputs))]
answers = [llama_outputs[i]['answer'] for i in range(len(llama_outputs))]
questions = [llama_outputs[i]['question'] for i in range(len(llama_outputs))]
bleus = [llama_outputs[i]['bleu'] for i in range(len(llama_outputs))]

res = []
for i in range(len(responses)):
    question = questions[i]
    response = responses[i]
    answer = answers[i]
    bleu = bleus[i]
    match = response == answer
    res.append({'question' : question, 'response' : response, 'answer' : answer, 'bleu' : bleu, 'match' : match})

    
write_jsonl(save_path, res)